# FAKE NEWS PREDICTION

### About the Dataset:
id: unique id for a news article
title: the title of a news article
author: author of the news article
text: the text of the article; could be incomplete
label: a label that marks whether the news article is real or fake



## Importing the dependencies

In [2]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Printing the stopwords

In [4]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

## Preprocessing

In [5]:
# Loading the dataset to pandas data frame
news_dataset = pd.read_csv('news.csv')

In [6]:
news_dataset.shape

(6335, 4)

In [7]:
# print the first 5 rows of the dataframe
news_dataset.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [8]:
# counting the number of missing values in the dataset
news_dataset.isnull().sum()

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64

In [9]:
# Replacing column text to content
news_dataset.rename(columns={'text': 'content'}, inplace=True)

In [10]:
print(news_dataset['content'])

0       Daniel Greenfield, a Shillman Journalism Fello...
1       Google Pinterest Digg Linkedin Reddit Stumbleu...
2       U.S. Secretary of State John F. Kerry said Mon...
3       — Kaydee King (@KaydeeKing) November 9, 2016 T...
4       It's primary day in New York and front-runners...
                              ...                        
6330    The State Department told the Republican Natio...
6331    The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
6332     Anti-Trump Protesters Are Tools of the Oligar...
6333    ADDIS ABABA, Ethiopia —President Obama convene...
6334    Jeb Bush Is Suddenly Attacking Trump. Here's W...
Name: content, Length: 6335, dtype: object


In [11]:
# separating the data & label
X = news_dataset.drop(columns='label', axis=1)
Y = news_dataset['label']

In [12]:
print(X)
print(Y)

      Unnamed: 0                                              title  \
0           8476                       You Can Smell Hillary’s Fear   
1          10294  Watch The Exact Moment Paul Ryan Committed Pol...   
2           3608        Kerry to go to Paris in gesture of sympathy   
3          10142  Bernie supporters on Twitter erupt in anger ag...   
4            875   The Battle of New York: Why This Primary Matters   
...          ...                                                ...   
6330        4490  State Department says it can't find emails fro...   
6331        8062  The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...   
6332        8622  Anti-Trump Protesters Are Tools of the Oligarc...   
6333        4021  In Ethiopia, Obama seeks progress on peace, se...   
6334        4330  Jeb Bush Is Suddenly Attacking Trump. Here's W...   

                                                content  
0     Daniel Greenfield, a Shillman Journalism Fello...  
1     Google Pinterest Digg Lin

## Stemming

In [13]:
port_stem = PorterStemmer()

In [14]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [15]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [16]:
print(news_dataset['content']) 

0       daniel greenfield shillman journal fellow free...
1       googl pinterest digg linkedin reddit stumbleup...
2       u secretari state john f kerri said monday sto...
3       kayde king kaydeek novemb lesson tonight dem l...
4       primari day new york front runner hillari clin...
                              ...                        
6330    state depart told republican nation committe c...
6331    p pb stand plutocrat pentagon post oct wikimed...
6332    anti trump protest tool oligarchi reform alway...
6333    addi ababa ethiopia presid obama conven meet l...
6334    jeb bush suddenli attack trump matter jeb bush...
Name: content, Length: 6335, dtype: object


In [17]:
#separating the data and label
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [18]:
print(X)

['daniel greenfield shillman journal fellow freedom center new york writer focus radic islam final stretch elect hillari rodham clinton gone war fbi word unpreced thrown around often elect ought retir still unpreced nomine major polit parti go war fbi exactli hillari peopl done coma patient wake watch hour cnn hospit bed would assum fbi director jame comey hillari oppon elect fbi attack everyon obama cnn hillari peopl circul letter attack comey current media hit piec lambast target trump surpris clinton alli start run attack ad fbi fbi leadership warn entir left wing establish form lynch mob continu go hillari fbi credibl attack media democrat preemptiv head result investig clinton foundat hillari clinton covert struggl fbi agent obama doj peopl gone explos public new york time compar comey j edgar hoover bizarr headlin jame comey role recal hoover fbi fairli practic admit front spout nonsens boston globe publish column call comey resign outdon time editori claim scandal realli attack 

In [19]:
print(Y)

['FAKE' 'FAKE' 'REAL' ... 'FAKE' 'REAL' 'REAL']


In [20]:
Y.shape

(6335,)

In [21]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [22]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1598597 stored elements and shape (6335, 43733)>
  Coords	Values
  (0, 106)	0.02761351227547292
  (0, 212)	0.023325671822305483
  (0, 271)	0.054392547007153
  (0, 328)	0.060358339691341766
  (0, 357)	0.013921754610837712
  (0, 451)	0.020578335403474236
  (0, 578)	0.07906058794712593
  (0, 617)	0.019850889079147057
  (0, 620)	0.01867769280554434
  (0, 622)	0.044611051373187424
  (0, 648)	0.015493583340569357
  (0, 680)	0.019224695171899866
  (0, 978)	0.0267316264615063
  (0, 999)	0.020738114075644642
  (0, 1022)	0.0371485992478585
  (0, 1033)	0.014808690907292291
  (0, 1085)	0.028815013247480384
  (0, 1089)	0.018102417756284926
  (0, 1197)	0.022505695388032227
  (0, 1206)	0.010675511636980443
  (0, 1549)	0.02507693986049084
  (0, 1665)	0.025967702221880915
  (0, 1713)	0.041128098634750046
  (0, 1744)	0.027618563562476325
  (0, 1783)	0.018872240981032625
  :	:
  (6334, 39650)	0.02398512016423436
  (6334, 39971)	0.0407182753213

In [23]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

## Training the Model: Logistic Regression

In [24]:
model = LogisticRegression()

In [25]:
model.fit(X_train, Y_train)

LogisticRegression()

## Evaluation
### Accuracy score

In [26]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [27]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9518547750591949


In [28]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [29]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.9163378058405682


## Making a Predictive System

In [30]:
X_new = X_test[3]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

['FAKE']
The news is Fake


In [31]:
print(Y_test[3])

FAKE


In [43]:
def predict_custom_news(news_text):
    # Preprocess like training data
    port_stem = PorterStemmer()
    cleaned = re.sub('[^a-zA-Z]', ' ', news_text)
    cleaned = cleaned.lower()
    cleaned = cleaned.split()
    cleaned = [port_stem.stem(word) for word in cleaned if word not in stopwords.words('english')]
    cleaned = ' '.join(cleaned)

    # Vectorize the cleaned input
    vector_input = vectorizer.transform([cleaned]).toarray()

    # Predict using trained model
    prediction = model.predict(vector_input)[0]

    # Print result
    if prediction == 'REAL':
        print("The news is Real.")
    else:
        print("The news is Fake.")


In [44]:
predict_custom_news("Hillary Clinton allies begin an unprecedented attack on the FBI just days before the election, suggesting bias in the bureau's investigation. This move reflects panic within the campaign.")

The news is Fake.


In [45]:
predict_custom_news("""U.S. Secretary of State John Kerry met with French President François Hollande following the unity march in Paris. 
Kerry emphasized the United States' strong support for France in the wake of the terror attacks. 
He announced plans for increased counter-terrorism cooperation and intelligence sharing between the two countries.""")

The news is Real.


In [47]:
predict_custom_news("""Newly leaked emails reveal that Hillary Clinton's campaign coordinated with the FBI to suppress an investigation into her private server. 
Sources close to the campaign confirmed that classified information was deleted intentionally to avoid criminal charges. 
This suggests a massive cover-up involving top Democratic officials and foreign governments.
""")

The news is Fake.


In [48]:
import pickle

# Save the model
with open('fake_news_model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

# Save the vectorizer
with open('tfidf_vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump(vectorizer, vectorizer_file)
